In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import geopandas as gpd

In [ ]:

# Step 1: Read data and calculate dependency weights
def calculate_dependency_weights(df):
    """
    Calculate dependency weights w_ij = t_ij / Σ_j t_ij
    where t_ij is the commuting flow from i to j
    """
    # Calculate total outflow for each origin city
    total_outflow = df.groupby('ORIGIN_TTWA')['TOTAL_FLOW'].sum()
    
    # Calculate dependency weights
    df['dependency_weight'] = df.apply(
        lambda row: row['TOTAL_FLOW'] / total_outflow[row['ORIGIN_TTWA']], 
        axis=1
    )
    
    return df

# Read data
df = pd.read_csv('ttwa_od_matrix_cross_city_only.csv')
df_with_weights = calculate_dependency_weights(df)

print(f"Data overview: {len(df)} OD records, {df['ORIGIN_TTWA'].nunique()} origin cities")
print(f"Dependency weight range: {df_with_weights['dependency_weight'].min():.6f} - {df_with_weights['dependency_weight'].max():.6f}")

In [ ]:
df_with_weights.head()

In [ ]:
# Step 2: Build directed commuting network
def build_commuting_network(df_weights):
    """Build directed commuting network"""
    G = nx.DiGraph()
    
    # Add edges with dependency weights
    for _, row in df_weights.iterrows():
        G.add_edge(
            row['ORIGIN_TTWA'], 
            row['DEST_TTWA'], 
            weight=row['dependency_weight'],
            flow=row['TOTAL_FLOW']
        )
    
    return G

G = build_commuting_network(df_with_weights)
print(f"Network scale: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

In [ ]:
# Load TTWA data
ttwa_gdf = gpd.read_file("boundary/Travel_to_Work_Areas_Dec_2011_FCB_in_United_Kingdom_2022.geojson")
ttwa_gdf = ttwa_gdf.to_crs('EPSG:4326')
ttwa_gdf = ttwa_gdf[['TTWA11NM', 'geometry']]

In [ ]:
import json
import folium
from itertools import combinations

# Calculate city centroid coordinates
city_centroids = {}
for idx, row in ttwa_gdf.iterrows():
    city_name = row['TTWA11NM']
    centroid = row['geometry'].centroid
    city_centroids[city_name] = (centroid.y, centroid.x)  # (lat, lon)

print(f"Number of city centroids: {len(city_centroids)}")
print("First 5 city centroids:")
for i, (city, coords) in enumerate(city_centroids.items()):
    if i < 5:
        print(f"  {city}: {coords}")

###### Currently in use

In [ ]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import os
import random

# ============================================================================
# Preparation: Assume the following variables are already prepared in your environment
# G: Your original NetworkX DiGraph
# ttwa_gdf: Your GeoDataFrame with geographic boundary data
# city_centroids: Your city centroid coordinate dictionary {'city_name': (latitude, longitude)}
# ============================================================================

def get_color_palette(num_colors):
    """
    Generate a high-quality, visually distinct color list.
    If the required number of colors exceeds the preset list, generate pseudo-random but distinct colors by sampling HSV space.
    """
    colors_tab20 = list(plt.cm.get_cmap('tab20').colors)
    colors_tab20b = list(plt.cm.get_cmap('tab20b').colors)
    colors_set3 = list(plt.cm.get_cmap('Set3').colors)
    # Combine into a larger high-quality color list, total 20 + 20 + 12 = 52 colors
    combined_colors = colors_tab20 + colors_tab20b + colors_set3

    if num_colors > len(combined_colors):
        print(f"  -> Warning: Number of predicted communities({num_colors}) > total preset colors({len(combined_colors)}). Will generate additional random colors.")
        # Use random generation to supplement colors to avoid duplication
        for i in range(num_colors - len(combined_colors)):
            combined_colors.append((random.random(), random.random(), random.random()))
    
    return combined_colors


def plot_network_with_color_inheritance(
    G, ttwa_gdf, city_centroids, threshold, output_folder, color_state
):
    """
    Use Matplotlib to plot high-quality static network OD diagram with color inheritance.
    (Final optimized version v4: implements color inheritance logic)
    
    Args:
        G (nx.DiGraph): Original network graph.
        ttwa_gdf (gpd.GeoDataFrame): Geographic boundary data.
        city_centroids (dict): City centroid coordinates.
        threshold (float): Current weight threshold.
        output_folder (str): Image output folder.
        color_state (dict): A dictionary containing color assignment state for cross-threshold tracking.

    Returns:
        dict: Updated color_state for next iteration.
    """
    print(f"\nStarting processing threshold τ = {threshold:.6f}")

    # --- 1. Filter network (logic unchanged) ---
    G_filtered = nx.Graph()
    for u, v, d in G.edges(data=True):
        if d.get('weight', 0) >= threshold:
            if not G_filtered.has_edge(u, v) or d['weight'] > G_filtered[u][v]['weight']:
                G_filtered.add_edge(u, v, weight=d['weight'])

    if G_filtered.number_of_edges() == 0:
        print(f"  -> No edges meet criteria at this threshold, skipping plot.")
        return color_state # Return current state directly without modification

    # --- 2. Core change: Community detection and color inheritance logic ---
    print("  -> Identifying communities and executing color inheritance logic...")
    # Find all connected components (communities) at current threshold
    current_components = list(nx.connected_components(G_filtered))
    current_components.sort(key=len, reverse=True)
    num_components = len(current_components)
    print(f"  -> Found {num_components} connected components.")

    # Get historical information from state dictionary
    cluster_color_map = color_state["cluster_color_map"]
    previous_node_to_cluster = color_state["previous_node_to_cluster"]
    previous_cluster_sizes = color_state["previous_cluster_sizes"]
    color_generator = color_state["color_generator"]

    # Prepare storage for current iteration information
    current_node_to_cluster = {}
    current_cluster_sizes = {}
    component_colors = {} # Color mapping for current plot

    # Iterate through all current communities to assign identities and colors
    for component in current_components:
        # Use the lexicographically smallest node name in the community as the unique, stable "identity ID"
        canonical_id = min(component)
        
        # Find the "parent" community of this community
        # That is, see which communities the member nodes of this community belonged to in the previous threshold
        parent_ids = {previous_node_to_cluster.get(node) for node in component}
        parent_ids.discard(None) # Remove nodes that didn't belong to any community before

        assigned_color = None
        if not parent_ids:
            # Case 1: This is a completely new community (all members were not in any community before)
            # Check if this community ID already has a color (unlikely but for robustness), otherwise get a new color from the generator
            if canonical_id not in cluster_color_map:
                try:
                    cluster_color_map[canonical_id] = next(color_generator)
                except StopIteration:
                    print("  -> Warning: Preset colors exhausted, will use random colors.")
                    cluster_color_map[canonical_id] = (random.random(), random.random(), random.random())
        else:
            # Case 2: This is a community merged/grown from one or more old communities
            # Find the largest "parent" community and inherit its color
            largest_parent_id = max(parent_ids, key=lambda pid: previous_cluster_sizes.get(pid, 0))
            cluster_color_map[canonical_id] = cluster_color_map[largest_parent_id]
            
        # Record colors needed for current plot
        component_colors[canonical_id] = cluster_color_map[canonical_id]

        # Update current node to community ID mapping and record community size
        current_cluster_sizes[canonical_id] = len(component)
        for node in component:
            current_node_to_cluster[node] = canonical_id

    # --- 3. Prepare geographic plotting data (using new color logic) ---
    ttwa_gdf['component_id'] = ttwa_gdf['TTWA11NM'].map(current_node_to_cluster)
    ttwa_gdf['color'] = ttwa_gdf['component_id'].map(component_colors)
    ttwa_gdf_filtered = ttwa_gdf[ttwa_gdf['component_id'].notna()].copy()

    # --- 4. Use Matplotlib for plotting (plotting code basically unchanged) ---
    fig, ax = plt.subplots(1, 1, figsize=(12, 12), facecolor='white')
    ax.set_aspect('equal')
    ttwa_gdf.plot(ax=ax, color='#f0f0f0', edgecolor='white', linewidth=0.5)

    if not ttwa_gdf_filtered.empty:
        ttwa_gdf_filtered.plot(
            ax=ax,
            color=ttwa_gdf_filtered['color'],
            edgecolor='white',
            linewidth=0.5,
            alpha=0.6
        )

    for u, v, d in G_filtered.edges(data=True):
        if u in city_centroids and v in city_centroids:
            lon1, lat1 = city_centroids[u][1], city_centroids[u][0]
            lon2, lat2 = city_centroids[v][1], city_centroids[v][0]
            ax.plot([lon1, lon2], [lat1, lat2],
                    color='dimgray', alpha=0.7, linewidth=1.0, zorder=2)

    node_lons = [city_centroids[node][1] for node in G_filtered.nodes() if node in city_centroids]
    node_lats = [city_centroids[node][0] for node in G_filtered.nodes() if node in city_centroids]
    node_colors_for_plot = [component_colors[current_node_to_cluster[node]] for node in G_filtered.nodes() if node in city_centroids and node in current_node_to_cluster]

    if node_lons: # Ensure there are nodes to plot
        ax.scatter(
            node_lons,
            node_lats,
            color=node_colors_for_plot,
            s=12,
            edgecolor='white',
            linewidth=0.75,
            zorder=3
        )
    
    # --- 5. Set map layout and style (logic unchanged) ---
    ax.axis('off')
    ax.text(0.02, 0.02, f'τ = {threshold:.6f}',
            transform=ax.transAxes,
            fontsize=14,
            verticalalignment='bottom',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7, edgecolor='none'))

    # --- 6. Save image (logic unchanged) ---
    base_filename = f"static_network_map_threshold_{threshold:.6f}.png"
    full_path = os.path.join(output_folder, base_filename)
    plt.savefig(full_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close(fig)
    print(f"  -> Static map saved to: {full_path}")
    
    # --- 7. Update and return state for next iteration preparation ---
    color_state["previous_node_to_cluster"] = current_node_to_cluster
    color_state["previous_cluster_sizes"] = current_cluster_sizes
    # cluster_color_map and color_generator are continuously updated throughout the process
    return color_state


# ============================================================================
# Main program: Use new color inheritance logic for plotting
# ============================================================================

# Your provided thresholds (τ)
thresholds_tau = [
    0.2632314021336898, 0.2450320844545643, 0.23822027229825488, 0.22633390705679862, 0.21169253597723733, 0.20324210418351815, 0.19528607869616907, 0.19174306211207864, 0.1826162604079985
]
# Ensure thresholds are processed from high to low, which is a prerequisite for color inheritance logic to work correctly
thresholds_tau.sort(reverse=True)

# Define and create output folder
output_directory = "static_maps_output_v4_inheritance"
os.makedirs(output_directory, exist_ok=True)
print(f"All generated images will be saved in '{output_directory}' folder.")

# --- Initialize color state manager ---
# Pre-estimate the maximum number of colors that might be needed, here we assume it won't exceed the total number of nodes in the graph
# This is a relatively safe upper bound that can avoid regenerating color lists during the loop
max_possible_colors = G.number_of_nodes() if 'G' in locals() else 500 # Use node count if G exists
color_palette = get_color_palette(max_possible_colors)

# color_state will be passed and updated in each iteration of the loop
color_state = {
    "cluster_color_map": {},          # Core: stores each community identity ID -> color mapping
    "previous_node_to_cluster": {}, # Stores previous iteration: each node -> its community ID mapping
    "previous_cluster_sizes": {},   # Stores previous iteration: each community ID -> its size mapping
    "color_generator": iter(color_palette) # A one-time color generator ensuring no color duplication
}

print(f"\nStarting to plot {len(thresholds_tau)} threshold static network graphs using new color inheritance logic...")

# Ensure G, ttwa_gdf, city_centroids are ready
for tau_threshold in thresholds_tau:
#     # Call new plotting function and receive updated state
     color_state = plot_network_with_color_inheritance(
         G, ttwa_gdf, city_centroids, tau_threshold, output_directory, color_state
    )

print("\nAll plotting tasks completed!")
print("Note: You need to integrate this code into your own environment,")
print("   and uncomment the main loop above, ensuring variables G, ttwa_gdf, city_centroids are properly loaded.")

————————

In [ ]:
list(G.edges(data=True))

In [ ]:
ttwa_gdf.head()

In [ ]:
# Load TTWA data
ttwa_gdf = gpd.read_file("boundary/Travel_to_Work_Areas_Dec_2011_FCB_in_United_Kingdom_2022.geojson")
ttwa_gdf = ttwa_gdf.to_crs('EPSG:4326')
ttwa_gdf = ttwa_gdf[['TTWA11NM', 'geometry']]

In [ ]:
import json
import folium
from itertools import combinations

# Calculate city centroid coordinates
city_centroids = {}
for idx, row in ttwa_gdf.iterrows():
    city_name = row['TTWA11NM']
    centroid = row['geometry'].centroid
    city_centroids[city_name] = (centroid.y, centroid.x)  # (lat, lon)

print(f"Number of city centroids: {len(city_centroids)}")
print("First 5 city centroids:")
for i, (city, coords) in enumerate(city_centroids.items()):
    if i < 5:
        print(f"  {city}: {coords}")